In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
iris_path = "/content/drive/MyDrive/ML/Lab 8/iris.csv"
df = pd.read_csv(iris_path)

In [3]:
y = df['variety']
X = df.drop(['variety'], axis = 1)

In [4]:
import sklearn
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [61]:
class KNN:
  def __init__(self):
    self.X_train, self.y_train = [], []
  def fit(self, X, y):
    self.X_train = X.reset_index().drop(['index'], axis = 1)
    self.Y_train = y
  def distance(self, x1, x2):
    dst = 0
    #print(list(x1),list(x2))
    for i in range(4):
      dst += (list(x1)[i] - list(x2)[i]) ** 2
    return dst
  def mode(self, arr):
    dic = {"Virginica": 0, "Setosa": 0, "Versicolor": 0}
    #print(arr)
    for i in arr:
      dic[list(self.Y_train)[i[1]]] += 1
    return max(dic, key = dic.get)
  def predict(self, X, k):
    distances = []
    preds = []
    for i1, i in X.iterrows():
      d = []
      for j1, j in self.X_train.iterrows():
        d.append((self.distance(i, j), j1))
      s = sorted(d)
      distances.append(s)
      preds.append(self.mode(s[:k]))
    return preds
model = KNN()
model.fit(X_train, y_train)
y_preds = model.predict(X_test, 3)

In [59]:
print(list(y_test))
print(y_preds)

['Virginica', 'Versicolor', 'Setosa', 'Virginica', 'Setosa', 'Virginica', 'Setosa', 'Versicolor', 'Versicolor', 'Versicolor', 'Virginica', 'Versicolor', 'Versicolor', 'Versicolor', 'Versicolor', 'Setosa', 'Versicolor', 'Versicolor', 'Setosa', 'Setosa', 'Virginica', 'Versicolor', 'Setosa', 'Setosa', 'Virginica', 'Setosa', 'Setosa', 'Versicolor', 'Versicolor', 'Setosa']
['Virginica', 'Versicolor', 'Setosa', 'Virginica', 'Setosa', 'Virginica', 'Setosa', 'Versicolor', 'Versicolor', 'Versicolor', 'Virginica', 'Versicolor', 'Versicolor', 'Versicolor', 'Virginica', 'Setosa', 'Versicolor', 'Versicolor', 'Setosa', 'Setosa', 'Virginica', 'Versicolor', 'Setosa', 'Setosa', 'Virginica', 'Setosa', 'Setosa', 'Versicolor', 'Versicolor', 'Setosa']


In [62]:
corr, tot = 0, len(y_test)
for i, j in zip(list(y_test), list(y_preds)):
  if i == j:
    corr += 1
print("Testing Accuracy:", corr / tot)
print("\nConfusion Matrix:\n", pd.crosstab(pd.Series(list(y_test)), pd.Series(y_preds)))

Testing Accuracy: 0.9666666666666667

Confusion Matrix:
 col_0       Setosa  Versicolor  Virginica
row_0                                    
Setosa          11           0          0
Versicolor       0          12          1
Virginica        0           0          6


In [64]:
conf_mat = pd.crosstab(pd.Series(list(y_test)), pd.Series(y_preds))

In [93]:
def macro_avg(conf_mat):
  precision, recall = 0, 0
  for i in range(len(conf_mat)):
    tp = conf_mat.iloc[i, i]
    row = sum(list(conf_mat.iloc[i, :]))
    col = sum(list(conf_mat.iloc[:, i]))
    precision += tp / row
    recall += tp / col
  return(precision / 3, recall / 3)
macro_precision, macro_recall = macro_avg(conf_mat)
print("Macro Precision:", macro_precision, "Macro Recall:", macro_recall)

Macro Precision: 0.9743589743589745 Macro Recall: 0.9523809523809524


In [94]:
def micro_avg(conf_mat):
  precision, recall = 0, 0
  tp, fp, fn = 0, 0, 0
  for i in range(len(conf_mat)):
    tp += conf_mat.iloc[i, i]
    fp += sum(list(conf_mat.iloc[i, :])) - conf_mat.iloc[i, i]
    fn += sum(list(conf_mat.iloc[:, i])) - conf_mat.iloc[i, i]
  return(tp / (tp + fp), tp / (tp + fn))
micro_precision, micro_recall = micro_avg(conf_mat)
print("Micro Precision:", micro_precision, "Micro Recall:", micro_recall)

Micro Precision: 0.9666666666666667 Micro Recall: 0.9666666666666667
